<a href="https://colab.research.google.com/github/luisddd8/Parcial3DataScience/blob/main/Proyecto_Tercer_Parcial_Data_Science_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
with open("laodisea.txt", "r", encoding="utf-8") as f:
    laodisea_text = f.read()

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(laodisea_text)

In [ ]:
tokenizer.texts_to_sequences(["First"])

[[30, 9, 8, 5, 12]]

In [ ]:
tokenizer.sequences_to_texts([[30, 9, 8, 5, 12]])

['f i r s t']

In [ ]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [ ]:
print(max_id)
print(dataset_size)

168
1001702


In [ ]:
[encoded] = np.array(tokenizer.texts_to_sequences([laodisea_text])) - 1

In [ ]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [ ]:
n_steps = 100
window_lenght = n_steps + 1
dataset = dataset.window(window_lenght, shift=1, drop_remainder=True)

In [ ]:
dataset = dataset.flat_map(lambda window: window.batch(window_lenght))

In [ ]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [ ]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch)
)
dataset = dataset.prefetch(1)

In [ ]:
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts))-1
  return tf.one_hot(X, max_id)

In [ ]:
def next_char(text, temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
def complete_text(text, n_chars=50, temperature=1):
  for _ in range(n_chars):
    text += next_char(text, temperature=1)
  return text

In [ ]:
# Si ya se entrenó el modelo y se tiene guardado en "modelo_stateless.keras" no se corre.

tf.random.set_seed(42)
batch_size = 32
encoded_parts = np.array_split(encoded[:train_size], batch_size)
datasets = []
for encoded_part in encoded_parts:
  dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
  dataset = dataset.window(window_lenght, shift=n_steps, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_lenght))
  datasets.append(dataset)
dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [ ]:
# Si ya se entrenó el modelo y se tiene guardado en "modelo_stateless.keras" no se corre.

model = keras.models.Sequential([
    keras.Input(batch_shape=(batch_size, None, max_id)),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])

In [ ]:
# Si ya se entrenó el modelo y se tiene guardado en "modelo_stateless.keras" no se corre.

class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        for layer in self.model.layers:
            if hasattr(layer, "reset_states"):
                layer.reset_states()

In [ ]:
# Si ya se entrenó el modelo y se tiene guardado en "modelo_stateless.keras" no se corre.

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])
model.save("modelo_entrenado.keras")

Epoch 1/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 147s 497ms/step - loss: 2.3201
Epoch 2/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 140s 491ms/step - loss: 2.1515
Epoch 3/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 143s 496ms/step - loss: 2.0446
Epoch 4/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 140s 499ms/step - loss: 1.9687
Epoch 5/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 141s 497ms/step - loss: 1.9149
Epoch 6/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 142s 498ms/step - loss: 1.8719
Epoch 7/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 140s 498ms/step - loss: 1.8387
Epoch 8/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 142s 499ms/step - loss: 1.8097
Epoch 9/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 140s 499ms/step - loss: 1.7865
Epoch 10/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 140s 496ms/step - loss: 1.7658
Epoch 11/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 139s 495ms/step - loss: 1.7503
Epoch 12/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 142s 493ms/step - loss: 1.7355
Epoch 13/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 143s 496ms/step - loss: 1.7220
Epoch 14/50
281/281 ━━━━━━━━━━━━━━━━━━━━ 139s 495ms/step - loss: 1.7071
E

In [ ]:
# Si ya se entrenó el modelo y se tiene guardado en "modelo_stateless.keras" no se corre.

stateless_model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Si ya se entrenó el modelo y se tiene guardado en "modelo_stateless.keras" no se corre.

# En esta celda es donde se guarda "modelo_stateless.keras" para generar el texto

stateless_model.build(tf.TensorShape([None, None, max_id]))
stateless_model.set_weights(model.get_weights())
tf.random.set_seed(42)
stateless_model.save("modelo_stateless.keras")

In [ ]:
#Correr cuando ya se tenga el archivo "modelo_stateless.keras"
#Si es la primera vez que se entrena no se corre.

model = keras.models.load_model("modelo_stateless.keras")

In [ ]:
def next_char(text, temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0, -1:, :]
  rescaled_logits = tf.math.log(y_proba) / temperature
  char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=50, temperature=1):
  for _ in range(n_chars):
    text += next_char(text, temperature=1)
  return text

print(complete_text("a", n_chars=100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━